In [38]:
import pandas as pd
import numpy as np
from itertools import product
import os

ModuleNotFoundError: No module named 'data'

In [18]:
os.getcwd()

'/Users/manueldelatejera/Desktop/Maestria/2do Semestre/Sale_Proyection/notebooks'

In [19]:
icategories = pd.read_csv("../data/archive/item_categories_en.csv")
items = pd.read_csv("../data/archive/items_en.csv")
shops = pd.read_csv("../data/archive/shops_en.csv")
train = pd.read_csv("../data/archive/sales_train.csv")
test = pd.read_csv("../data/archive/test.csv")
sample = pd.read_csv("../data/archive/sample_submission.csv")

train: (2935849, 6)
test: (214200, 3)


In [23]:
train["date"] = pd.to_datetime(train["date"], format="%d.%m.%Y")
train["month"] = train["date_block_num"].astype(int)


In [24]:
#Limpieza de outliers

train = train[(train["item_price"] > 0) & (train["item_price"] < 100000)]
print(train.shape)


(2935847, 7)


In [26]:
monthly = (
    train
    .groupby(["date_block_num", "shop_id", "item_id"], as_index=False)
    .agg(item_cnt_month=("item_cnt_day", "sum"))
)

print(monthly.shape)
monthly.head()


(1609123, 4)


,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [28]:
monthly["item_cnt_month"] = monthly["item_cnt_month"].clip(0, 20)
monthly["item_cnt_month"].describe()


count    1.609123e+06
mean     2.022806e+00
std      2.577964e+00
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      2.000000e+01
Name: item_cnt_month, dtype: float64

In [31]:
index_cols = ["date_block_num", "shop_id", "item_id"]

grids = []
for month in monthly["date_block_num"].unique():
    cur = train[train["date_block_num"] == month]
    shops_m = cur["shop_id"].unique()
    items_m = cur["item_id"].unique()
    grids.append(
        pd.DataFrame(list(product([month], shops_m, items_m)), columns=index_cols)
    )

grid = pd.concat(grids, ignore_index=True)
print("grid:", grid.shape)
grid.head()


grid: (10913804, 3)


,date_block_num,shop_id,item_id
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564


In [33]:
data = grid.merge(monthly, on=index_cols, how="left")
data["item_cnt_month"] = data["item_cnt_month"].fillna(0).astype(np.float32)

print(data.shape)
data.head()


(10913804, 4)


,date_block_num,shop_id,item_id,item_cnt_month
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0


In [35]:
data = data.merge(items[["item_id", "item_category_id"]], on="item_id", how="left")

# sanity check:
data.isna().sum().head(10)


date_block_num        0
shop_id               0
item_id               0
item_cnt_month        0
item_category_id_x    0
item_category_id_y    0
dtype: int64

In [42]:
data.to_csv("../data/new/monthly_dataset.csv", index=False)
print("Guardado")



Guardado
